# Project 2 - CS 181
### Ward Anderson, Ben Brown, Jeff Stevens
### Dr. Kretchmar
### 10/22/18

{intro goes here}

In [58]:
import pandas as pd
import sqlalchemy as sa
import matplotlib.pyplot as plt
import json

#Include inflation data
inflation = pd.read_csv("conversionfactors.csv")

In [11]:
def getCreds(filename,subset,defaults={}):
    '''
    This function helps us connect to a database on hadoop2.  The
    userid password information is stored in a local file encoded
    in json format.  
    Inputs:
        filename is where json encoded userid/password information stored
        subset : dictionary key in json file
        defaults : alternative way to connect to database
    Returns subdictionary based on subset parameter.
    '''
    try:
        with open(filename,'r') as file:
            D = json.load(file)
            file.close()
            if D[subset]:
                return D[subset]
            else:
                return defaults
    except:
        return defaults
     
def db_setup(database):
    template = 'mysql+mysqlconnector://{}:{}@hadoop2.mathsci.denison.edu/{}'
    cstring = template.format(creds['user'],creds['password'],database)
    return cstring

creds = getCreds('creds.json','mysql',defaults = {'user':'steven_j1', 'password':'steven_j1'})

cstring = db_setup("lahman2016")
engine = sa.create_engine(cstring)
connection = engine.connect()

In [35]:
     #-#-#-#-#-#-#-#-#-#-#-#-#-#-#
########## CLEVELAND INDIANS ##########
     #-#-#-#-#-#-#-#-#-#-#-#-#-#-#

query_main = '''SELECT yearID, teamID, franchID, H/AB AS BA, ERA , W/(W+L) AS Winpct, FP
FROM Teams
WHERE franchID = "CLE";'''

getclesal = "SELECT yearID, AVG(salary) AS CleSalary \
FROM Salaries \
WHERE teamID = 'CLE' \
GROUP BY yearID \
ORDER BY yearID;"

In [13]:
#Converting Cleveland Indians query to DataFrame
result = connection.execute(query_main)
rows = result.fetchall()
df = pd.DataFrame(rows,columns = result.keys())
df.columns = ["Year","Team","Franchise","Batting Average","Earned Run Average", "Win Percentage","Fielding Percentage"]
df = df.drop(columns = ["Team","Franchise"], axis = 1)
df.to_csv('CLE.csv')

In [14]:
print("Statistics for The Cleveland Indians Baseball Team since 1901")
print(df)

Statistics for The Cleveland Indians Baseball Team since 1901
     Year Batting Average  Earned Run Average Win Percentage  \
0    1901          0.2713                4.12         0.3971   
1    1902          0.2895                3.28         0.5074   
2    1903          0.2650                2.73         0.5500   
3    1904          0.2601                2.22         0.5695   
4    1905          0.2551                2.85         0.4935   
5    1906          0.2794                2.09         0.5817   
6    1907          0.2409                2.26         0.5592   
7    1908          0.2390                2.02         0.5844   
8    1909          0.2409                2.40         0.4641   
9    1910          0.2444                2.88         0.4671   
10   1911          0.2821                3.36         0.5229   
11   1912          0.2733                3.30         0.4902   
12   1913          0.2681                2.54         0.5658   
13   1914          0.2447                3

In [34]:
     #-#-#-#-#-#-#-#-#-#-#-#-#-#-#
########## WASHINGTON NATIONALS ##########
     #-#-#-#-#-#-#-#-#-#-#-#-#-#-#
    
    
WAS_query = '''SELECT t.yearID, t.teamID, t.franchID, t.H, t.AB, t.W, t.L, t.ERA, t.H/t.AB AS BA, t.W/(t.W+t.L) AS Winpct, t.FP
FROM Teams AS t
WHERE t.franchID = "WSN";'''
resultproxy = connection.execute(WAS_query)

getwsnsal = "SELECT yearID, AVG(salary) AS WsnSalary \
FROM Salaries \
WHERE teamID = 'WSN' \
OR teamID = 'WAS' \
OR teamID = 'MON' \
GROUP BY yearID \
ORDER BY yearID;"

In [16]:
#Main query
tmp = connection.execute(WAS_query)
WAS_stats = pd.DataFrame(tmp.fetchall(), columns = tmp.keys())
print(WAS_stats)

    yearID teamID franchID     H    AB   W    L   ERA      BA  Winpct     FP
0     1969    MON      WSN  1300  5419  52  110  4.33  0.2399  0.3210  0.970
1     1970    MON      WSN  1284  5411  73   89  4.50  0.2373  0.4506  0.977
2     1971    MON      WSN  1312  5335  71   90  4.12  0.2459  0.4410  0.976
3     1972    MON      WSN  1205  5156  70   86  3.59  0.2337  0.4487  0.978
4     1973    MON      WSN  1345  5369  79   83  3.71  0.2505  0.4877  0.974
5     1974    MON      WSN  1355  5343  79   82  3.60  0.2536  0.4907  0.976
6     1975    MON      WSN  1346  5518  75   87  3.72  0.2439  0.4630  0.973
7     1976    MON      WSN  1275  5428  55  107  3.99  0.2349  0.3395  0.975
8     1977    MON      WSN  1474  5675  75   87  4.01  0.2597  0.4630  0.980
9     1978    MON      WSN  1404  5530  76   86  3.42  0.2539  0.4691  0.979
10    1979    MON      WSN  1445  5465  95   65  3.14  0.2644  0.5938  0.979
11    1980    MON      WSN  1407  5465  90   72  3.48  0.2575  0.5556  0.977

In [33]:
     #-#-#-#-#-#-#-#-#-#-#-#-#-#-#
########## CINCINNATI REDS ##########
     #-#-#-#-#-#-#-#-#-#-#-#-#-#-#

getredstats = "SELECT t.yearID, t.teamID, t.franchID, t.H, t.AB, t.W, t.L, t.ERA, t.H/t.AB AS BA, t.W/(t.W+t.L) AS Winpct, t.FP \
FROM Teams AS t \
WHERE t.franchID = 'CIN';"

getredsal = "SELECT yearID, AVG(salary) AS RedsSalary \
FROM Salaries \
WHERE teamID = 'CIN' \
GROUP BY yearID \
ORDER BY yearID;"

tmp = connection.execute(getredstats)
redstats = pd.DataFrame(tmp.fetchall(), columns = tmp.keys())

print(redstats)

     yearID teamID franchID     H    AB   W   L   ERA      BA  Winpct     FP
0      1882    CN2      CIN   795  3007  55  25  1.65  0.2644  0.6875  0.903
1      1883    CN2      CIN   961  3669  61  37  2.26  0.2619  0.6224  0.908
2      1884    CN2      CIN  1037  4090  68  41  3.33  0.2535  0.6239  0.911
3      1885    CN2      CIN  1046  4050  63  49  3.26  0.2583  0.5625  0.911
4      1886    CN2      CIN  1225  4915  65  73  4.18  0.2492  0.4710  0.905
5      1887    CN2      CIN  1285  4797  81  54  3.58  0.2679  0.6000  0.915
6      1888    CN2      CIN  1161  4801  80  54  2.73  0.2418  0.5970  0.923
7      1889    CN2      CIN  1307  4844  76  63  3.50  0.2698  0.5468  0.926
8      1890    CIN      CIN  1204  4644  77  55  2.79  0.2593  0.5833  0.933
9      1891    CIN      CIN  1158  4791  56  81  3.55  0.2417  0.4088  0.931
10     1892    CIN      CIN  1288  5349  82  68  3.17  0.2408  0.5467  0.939
11     1893    CIN      CIN  1195  4617  65  63  4.55  0.2588  0.5078  0.943

In [75]:
#Aggregate salary data
tmp = connection.execute(getredsal)
redsal = pd.DataFrame(tmp.fetchall(), columns = tmp.keys())
tmp = connection.execute(getclesal)
clesal = pd.DataFrame(tmp.fetchall(), columns = tmp.keys())
tmp = connection.execute(getwsnsal)
wsnsal = pd.DataFrame(tmp.fetchall(), columns = tmp.keys())

salary = redsal.merge(clesal, on = "yearID").merge(wsnsal, on = "yearID").merge(inflation, left_on = "yearID", right_on = "Year")
salary = salary.drop("Year", axis = 1)
salary.RedsSalary = pd.to_numeric(salary.RedsSalary)
salary.CleSalary = pd.to_numeric(salary.CleSalary)
salary.WsnSalary = pd.to_numeric(salary.WsnSalary)

#Adjust for inflation
salary.RedsSalary = salary.RedsSalary/salary.Factor
salary.CleSalary = salary.CleSalary/salary.Factor
salary.WsnSalary = salary.WsnSalary/salary.Factor
salary = salary.drop("Factor", axis = 1)

print(salary)

    yearID    RedsSalary     CleSalary     WsnSalary
0     1985  8.655951e+05  7.462034e+05  1.078607e+06
1     1986  8.878738e+05  5.635780e+05  7.762584e+05
2     1987  7.710168e+05  6.567225e+05  6.247347e+05
3     1988  7.361001e+05  8.410032e+05  7.953071e+05
4     1989  7.545318e+05  6.656785e+05  9.745475e+05
5     1990  7.929588e+05  8.236398e+05  1.003836e+06
6     1991  1.819683e+06  1.057254e+06  1.135456e+06
7     1992  2.326567e+06  4.681840e+05  1.024497e+06
8     1993  2.305068e+06  7.864831e+05  8.213530e+05
9     1994  2.184049e+06  1.439929e+06  1.127391e+06
10    1995  2.040130e+06  1.742666e+06  6.023580e+05
11    1996  1.898497e+06  2.424766e+06  8.197833e+05
12    1997  1.899542e+06  2.797462e+06  9.502832e+05
13    1998  1.017470e+06  2.770570e+06  4.849168e+05
14    1999  1.611137e+06  2.824244e+06  7.522269e+05
15    2000  2.469164e+06  4.151481e+06  1.618401e+06
16    2001  2.509400e+06  4.294698e+06  1.568710e+06
17    2002  2.045885e+06  3.583535e+06  1.8815

## Plots